## FDA inspection data analytics - build the dataset

## load libraries

In [573]:
file = '../../data/input/raw_clinical_investigator_inspection_fc.xls'
file2 = '../../data/input/raw_clinical_investigator_inspection_da.xls'
file_map = '../../data/input/refined_mapping_fda_inspections_code.csv'
file_out = '../../data/output/final_set_FDA.csv'

In [574]:
import numpy as np
import os as os
import pandas as pd
import xlrd 


## load and clean fda inspections data

In [575]:
xl = pd.ExcelFile(file)

df_fc = xl.parse('Sheet1')


xl = pd.ExcelFile(file2)

df_da = xl.parse('Sheet1')

In [576]:
print(df_da.shape)
print (df_fc.shape)

(2500, 12)
(1774, 12)


In [577]:
# append the two df
df_insp = df_da.append(df_fc, ignore_index = True)

In [578]:
print(df_insp.shape)

(4274, 12)


In [579]:
df_insp

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,Insp. Class.,Insp. Def. Codes
0,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,08/18/00,DA,VAI,05
1,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,09/03/86,DA,VAI2,03
2,66,"Davidov, Michael E MD",-,311 Park Ave,Falls Church,VA,USA,22046,01/13/93,DA,NAI,00
3,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,12/07/15,DA,NAI,00
4,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,02/03/97,DA,VAI,03
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,719450,"Hassan, Kazi MD",Florida Medical Pain Management,6333 54Th Avenue North,St Petersburg,FL,USA,33709,03/05/19,FC,NAI,00
4270,719923,"Yi, Sung H MD",Advanced Research Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,00
4271,720510,"Huang, May",NaN,1221 Madison Street Suite 1410,Seattle,WA,USA,98104,11/15/18,FC,VAI,05
4272,721608,"Cho, Austina B MD",Advanced Reserch Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,00


In [580]:
# rename the 'Insp. Def. codes' to 'insp_codes' and 
df_insp.rename(columns={'Insp. Def. Codes':'insp_codes'}, inplace=True)
df_insp.rename(columns={'Insp. Class.':'insp_class'}, inplace=True)

In [581]:
# convert code '00' and '0' (=no finding noted) into '22' to avoid issues mixing up with NaN
df_insp['insp_codes'].replace(['00'], '22', inplace=True)
df_insp['insp_codes'].replace(['0'], '22', inplace=True)

In [582]:
# convert empty/blank cells in 'insp_codes'(=with no inspection data) into NaN
df_insp = df_insp.replace(r'^\s+$', np.nan, regex=True)

In [583]:
# drop washout inspections
df_insp1 = df_insp[df_insp.insp_class != 'WASH']
print(df_insp1.shape)

(4160, 12)


In [584]:
# drop canceled inspections
insp_data = df_insp1[df_insp1.insp_class != 'CANC']
print(insp_data.shape)

(3972, 12)


In [585]:
# look for inspections with no action required 'NAI' but w/o code assigned

insp_data_NULL = insp_data[insp_data['insp_codes'].isnull()]
print(insp_data_NULL.shape)

(937, 12)


In [586]:
insp_data_NAI_and_null = insp_data_NULL[insp_data_NULL['insp_class'] == 'NAI']
print(insp_data_NAI_and_null.shape)

(571, 12)


In [587]:
# there are inspections with insp_class = No Action Indicated (NAI) w/o inspection code
# assign them 'insp_code' = 22

insp_data.loc[(insp_data['insp_class'] == 'NAI'), 'insp_codes'] = '22'
print(insp_data.shape)

(3972, 12)


In [588]:
# check numbers add up

print((insp_data[insp_data['insp_codes'].isnull()]).shape)

# 961 rows with NaN = 639 insp_class = 'NAI' AND insp_code= '22' + 322 insp_code = NaN
# ---> OK

(366, 12)


In [589]:
# look for rows with typos
value_list = [',', ',,',',,,']

In [590]:
insp_data[insp_data.insp_codes.isin(value_list)]

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
520,14835,"Shalita, Alan R MD",Suny Hlth Sci Ctr ...,450 Clarkson Ave Box 46,Brooklyn,NY,USA,11203,08/08/83,DA,VAI3F,","
521,14835,"Shalita, Alan R MD",Suny Hlth Sci Ctr ...,450 Clarkson Ave Box 46,Brooklyn,NY,USA,11203,08/08/83,DA,VAI3R,","
1503,56699,"Scharf, Martin B PHD",Ctr Res Sleep Disorders/Tristate Sleep Disorde...,1275 E Kemper Rd,Cincinnati,OH,USA,45246,02/06/01,DA,OAIW,","
2297,83034,"Trapp, George A MD",Va Med Ctr,4500 S Lancaster Rd,Dallas,TX,USA,75216,10/16/80,DA,VAI2,","
2783,99809,"Moskowitz, George MD",-,101 Rutledge St,Charlestown,SC,USA,00000,05/29/81,FC,OAIC,","


In [591]:
# replace typo with '23' (=no data available)
insp_data['insp_codes'].replace([','], '23', inplace=True)

In [592]:
# replace typo (,00) - replace by 22
insp_data['insp_codes'].replace([',00'], '22', inplace=True)

In [593]:
insp_data

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
0,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,08/18/00,DA,VAI,05
1,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,09/03/86,DA,VAI2,03
2,66,"Davidov, Michael E MD",-,311 Park Ave,Falls Church,VA,USA,22046,01/13/93,DA,NAI,22
3,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,12/07/15,DA,NAI,22
4,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,02/03/97,DA,VAI,03
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,719450,"Hassan, Kazi MD",Florida Medical Pain Management,6333 54Th Avenue North,St Petersburg,FL,USA,33709,03/05/19,FC,NAI,22
4270,719923,"Yi, Sung H MD",Advanced Research Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22
4271,720510,"Huang, May",NaN,1221 Madison Street Suite 1410,Seattle,WA,USA,98104,11/15/18,FC,VAI,05
4272,721608,"Cho, Austina B MD",Advanced Reserch Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22


In [594]:
# explose insp_codes column (requires pandas >0.25)
insp_data_interim = insp_data.assign(insp_codes=df_insp.insp_codes.str.split(',')).explode('insp_codes')

In [595]:
# check
insp_data_interim

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes
0,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,08/18/00,DA,VAI,05
1,44,"Chey, William Y MD",Univ Rochester,601 Elmwood Ave Box 646,Rochester,NY,USA,14642,09/03/86,DA,VAI2,03
2,66,"Davidov, Michael E MD",-,311 Park Ave,Falls Church,VA,USA,22046,01/13/93,DA,NAI,22
3,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,12/07/15,DA,NAI,22
4,99,"Ciancio, Sebastian G DDS",State Univ @ New York Of Buffalo,3435 Main St/250 Squire Hall,Buffalo,NY,USA,14214,02/03/97,DA,VAI,03
...,...,...,...,...,...,...,...,...,...,...,...,...
4269,719450,"Hassan, Kazi MD",Florida Medical Pain Management,6333 54Th Avenue North,St Petersburg,FL,USA,33709,03/05/19,FC,NAI,22
4270,719923,"Yi, Sung H MD",Advanced Research Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22
4271,720510,"Huang, May",NaN,1221 Madison Street Suite 1410,Seattle,WA,USA,98104,11/15/18,FC,VAI,05
4272,721608,"Cho, Austina B MD",Advanced Reserch Center Inc,"1020 S Anaheim Blvd, Suite 316",Anaheim,CA,USA,92805,06/18/19,FC,NAI,22


In [596]:
# convert insp_code into numeric data for the join
insp_data_interim['insp_codes'] = pd.to_numeric(insp_data_interim['insp_codes'])

In [597]:
# Verify values in insp_codes
print(insp_data_interim.insp_codes.unique())

[ 5.  3. 22. 15.  6. nan  0.  4.  7. 16. 12. 14. 18.  1.  2.  8. 10. 13.
 11.  9. 17. 20. 19.]


In [598]:
# check 
print(insp_data_interim[insp_data_interim['insp_codes'].isnull()])

      ID No.                              Name  \
7        113   Vlachakis,   Nicholas   D    MD   
8        113   Vlachakis,   Nicholas   D    MD   
9        146      Homesley,   Howard   D    MD   
14       271          Lipton,   Allan       MD   
31       748    Pendergrass,   Kelly   B    MD   
...      ...                               ...   
4171  642028                Chisti,   Mohammad   
4178  648641                   Driver,   Brian   
4195  661444  Castellon,   Celestino   P    MD   
4201  666448                  Racke,   Michael   
4254  697186                Meltzer,   Herbert   

                                               Location  \
7                           Los Angeles Cty/Usc Med Ctr   
8                           Los Angeles Cty/Usc Med Ctr   
9                  Gynecology Oncology Middle Tennessee   
14                             Milton S Hershey Med Ctr   
31                             Kansas City Internal Med   
...                                          

In [599]:
#find duplicates

duplicateRowsDF = insp_data_interim[insp_data_interim.duplicated()]

print(duplicateRowsDF)


      ID No.                          Name  \
37       839      Arnold,   John   D    MD   
162     4682   Bodey,   Gerald   P  SR  MD   
162     4682   Bodey,   Gerald   P  SR  MD   
360    10639     Gentry,   Layne   O    MD   
360    10639     Gentry,   Layne   O    MD   
...      ...                           ...   
3874  499117  Montero,   Alberto   J    MD   
4041  542883      Renfroe,   Ben   J    MD   
4208  669694   Huffman,   Cynthia       MD   
4240  683889             Lang,   Jason   E   
4241  684207                Amin,   Faisal   

                                Location                              Address  \
37                                     -                       5104 E 24Th St   
162        Univ Texas Md Anderson Ca Ctr                   1515 Holcombe Blvd   
162        Univ Texas Md Anderson Ca Ctr                   1515 Holcombe Blvd   
360              St Lukes Episcopal Hosp                     6720 Bertner Ave   
360              St Lukes Episcopal Hosp  

In [600]:
# drop duplicates
insp_data_final = insp_data_interim.drop_duplicates(keep='first')

In [601]:
# confirm # of rows with NaN
print((insp_data_final[insp_data_final['insp_codes'].isnull()]).shape)

(1024, 12)


## load mapping data 

In [602]:
# load csv file - created and refined based on:
# https://www.fda.gov/drugs/enforcement-activities-fda/clinical-investigator-inspection-list-ciil-database-codes)
mapping = pd.read_csv (file_map, encoding = 'cp1250',low_memory=False)

In [627]:
# check
mapping.sort_values('deficiency')

,insp_codes,deficiency,cfr_reference,mapped_clinical_impact_factor
4,5,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety
11,12,Failure to list additional investigators on 1572,21 CFR 312.60,primary_endpoints
14,15,"Failure to notify IRB of changes, failure to s...",21 CFR 312.66,consent
1,2,Failure to obtain and/or document subject consent,"21 CFR 312.60, 50.20, 50.27",consent
13,14,Failure to obtain or document IRB approval,"21 CFR 312.60, 62, 66; 56.103",consent
20,21,Failure to permit FDA access to records,21 CFR 312.68,human_subject_protection
19,20,"Failure to protect the rights, safety, and wel...",21 CFR 312.60,human_subject_protection
15,16,Failure to report adverse drug reactions,"21 CFR 312.64, 312.66",safety
18,19,Failure to supervise or personally conduct the...,21 CFR 312.60,primary_endpoints_and_safety
5,6,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints


In [604]:
# rename column to insp_codes to allow mapping with FDA dataset
mapping.rename(columns={'code':'insp_codes'}, inplace=True)

In [605]:
# insp_codes to numeric value
mapping['insp_codes'] = pd.to_numeric(mapping['insp_codes'])

## join and clean consolidated data set ('fda_inspections')

In [606]:
# do the join
finalset = pd.merge (insp_data_final,mapping, on='insp_codes', how='left')

In [607]:
# check
print(finalset.shape)
finalset.info()

(7472, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7472 entries, 0 to 7471
Data columns (total 15 columns):
ID No.                           7472 non-null int64
Name                             7472 non-null object
Location                         7469 non-null object
Address                          7472 non-null object
City                             7472 non-null object
State                            7472 non-null object
Cntry                            7472 non-null object
Zip                              7472 non-null object
Insp. Date                       7455 non-null object
Insp. Type                       7472 non-null object
insp_class                       7472 non-null object
insp_codes                       6448 non-null float64
deficiency                       6421 non-null object
cfr_reference                    5847 non-null object
mapped_clinical_impact_factor    6421 non-null object
dtypes: float64(1), int64(1), object(13)
memory usage: 934.0+ KB


## final cleaning

In [608]:
print(finalset.insp_codes.unique())

[ 5.  3. 22. 15.  6. nan  0.  4.  7. 16. 12. 14. 18.  1.  2.  8. 10. 13.
 11.  9. 17. 20. 19.]


In [609]:
print(finalset[finalset['insp_codes'].isnull()])

      ID No.                              Name  \
11       113   Vlachakis,   Nicholas   D    MD   
12       113   Vlachakis,   Nicholas   D    MD   
13       146      Homesley,   Howard   D    MD   
26       271          Lipton,   Allan       MD   
57       748    Pendergrass,   Kelly   B    MD   
...      ...                               ...   
7342  642028                Chisti,   Mohammad   
7351  648641                   Driver,   Brian   
7372  661444  Castellon,   Celestino   P    MD   
7379  666448                  Racke,   Michael   
7449  697186                Meltzer,   Herbert   

                                               Location  \
11                          Los Angeles Cty/Usc Med Ctr   
12                          Los Angeles Cty/Usc Med Ctr   
13                 Gynecology Oncology Middle Tennessee   
26                             Milton S Hershey Med Ctr   
57                             Kansas City Internal Med   
...                                          

In [610]:
# Add insp_code '23' --> "no data reported"

finalset['insp_codes'].fillna(23, inplace=True)

In [611]:
# confirm no NaN
print((finalset[finalset['insp_codes'].isnull()]).shape)

(0, 15)


# Features

## Inverstigate ID

In [612]:
finalset['year'] = pd.to_datetime( finalset['Insp. Date'] ).dt.year

In [613]:
d1 = finalset.loc[:,['ID No.','year']] \
    .groupby('ID No.') \
    .aggregate('nunique') \
    .sort_values('year', ascending = False)
    
d2 = finalset.loc[:,['ID No.','year']] \
    .groupby('ID No.') \
    .aggregate(lambda x: set(x) ) 

pd.merge(d1, d2, left_index = True, right_index = True)

,ID No.,year_x,year_y
ID No.,,,
83603,1,9,"{1995.0, 1996.0, 1998.0, 2001.0, 2003.0, 2009...."
4364,1,8,"{1985.0, 1988.0, 1989.0, 1992.0, 1997.0, 2004...."
90608,1,8,"{1984.0, 1988.0, 1989.0, 1992.0, 1997.0, 2000...."
30728,1,7,"{1985.0, 1991.0, 1995.0, 1996.0, 1998.0, 1980...."
28875,1,7,"{1987.0, 1995.0, 1996.0, 2002.0, 2006.0, 2008...."
...,...,...,...
479617,1,0,{nan}
210402,1,0,{nan}
310292,1,0,{nan}


In [614]:
finalset.loc[ finalset['ID No.'] == 83603, : ]

,ID No.,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes,deficiency,cfr_reference,mapped_clinical_impact_factor,year
3740,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,10/31/96,DA,NAI,23.0,NaN,NaN,NaN,1996.0
3741,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,01/29/13,DA,VAI,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,2013.0
3742,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,01/29/13,DA,VAI,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,2013.0
3743,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,06/18/12,DA,VAI,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,2012.0
3744,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,06/18/12,DA,VAI,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,2012.0
3745,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,03/12/98,DA,NAI,23.0,NaN,NaN,NaN,1998.0
3746,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,06/01/09,DA,VAI2,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,2009.0
3747,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,06/01/09,DA,VAI2,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,2009.0
3748,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,02/11/03,DA,NAI,23.0,NaN,NaN,NaN,2003.0
3749,83603,"Jarratt, Michael T MD","Dermresearch, Inc.","8140 N Mopac Bldg. 3, Ste. 120",Austin,TX,USA,78759,12/13/82,DA,VAI2,2.0,Failure to obtain and/or document subject consent,"21 CFR 312.60, 50.20, 50.27",consent,1982.0


In [615]:
d1 = finalset.loc[:,['ID No.','Location', 'Name']] \
    .groupby('ID No.') \
    .aggregate('nunique') \
    .sort_values('Location', ascending = False)
    
d2 = finalset.loc[:,['ID No.','Location', 'Name']] \
    .groupby('ID No.') \
    .aggregate(lambda x: set(x) ) 

pd.merge(d1, d2, left_index = True, right_index = True)

,ID No.,Location_x,Name_x,Location_y,Name_y
ID No.,,,,,
44,1,1,1,{Univ Rochester},"{Chey, William Y MD}"
377718,1,1,1,{-},"{Russo, Carlo}"
377732,1,1,1,{-},"{Shroff, Phyllis F MD}"
377733,1,1,1,{Eastside Comprehensive Med Services},"{Siegel, Howard I MD}"
377735,1,1,1,{-},"{Simor, George MD}"
...,...,...,...,...,...
59541,1,1,1,{Med Arts Complex},"{Kennedy, Alastair C MD}"
59552,1,1,1,{Washington Cardiol Ctr},"{Kent, Kenneth M MD}"
59585,1,1,1,{Univ Utah Med Ctr},"{Krueger, Gerald G MD}"


The ID column refers to a site + Investigator combination

In [616]:
finalset.rename( {'ID No.' : 'site_id'}, axis = 1, inplace = True )

# Create Inspection ID

In [617]:
site_id = finalset['site_id']\
    .astype('str') \
    .str.pad(width = 6, fillchar= '0')

site_id

0       000044
1       000044
2       000066
3       000099
4       000099
         ...  
7467    719450
7468    719923
7469    720510
7470    721608
7471    722902
Name: site_id, Length: 7472, dtype: object

In [618]:
date_id = finalset['Insp. Date'].str.replace('/', '') 
date_id

0       081800
1       090386
2       011393
3       120715
4       020397
         ...  
7467    030519
7468    061819
7469    111518
7470    061819
7471    010719
Name: Insp. Date, Length: 7472, dtype: object

In [619]:
finalset['insp_id'] = site_id + date_id

In [620]:
finalset.query('insp_id =="080295081180"') \
    .sort_values('insp_codes')

,site_id,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes,deficiency,cfr_reference,mapped_clinical_impact_factor,year,insp_id
3384,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,1.0,Records availability,21 CFR 312.62,data_integrity,1980.0,080295081180
4501,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,1.0,Records availability,21 CFR 312.62,data_integrity,1980.0,080295081180
3385,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,3.0,Inadequate informed consent form,21 CFR 50.25,consent,1980.0,080295081180
4502,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,3.0,Inadequate informed consent form,21 CFR 50.25,consent,1980.0,080295081180
3386,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,1980.0,080295081180
4503,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,1980.0,080295081180
3387,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,1980.0,080295081180
4504,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,1980.0,080295081180
3388,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,1980.0,080295081180
4505,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,1980.0,080295081180


In [621]:
finalset['insp_class'] = finalset['insp_class'].fillna('NA')
finalset['Insp. Type'] = finalset['Insp. Type'].fillna('NA')

In [623]:
# df_gr = insp_class = finalset \
#     .loc[:,['insp_id','insp_class', 'Insp. Type']] \
#     .groupby('insp_id') \
#     .transform(lambda x: set(x))
# 
# for some reason transfom() throws an error so we have to create the sets manually

df_gr = insp_class = finalset \
    .loc[:,['insp_id','insp_class', 'Insp. Type']] \
    .groupby('insp_id') \
    .aggregate(lambda x: ','.join( list( set(x) ) ) )
    



In [624]:
finalset = finalset.drop(['insp_class', 'Insp. Type'], axis = 1) \
    .merge(df_gr, on = 'insp_id', how = 'left' ) \
    .drop_duplicates()



In [625]:
finalset.query('insp_id =="080295081180"') \
    .sort_values('insp_codes')

,site_id,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,insp_codes,deficiency,cfr_reference,mapped_clinical_impact_factor,year,insp_id,insp_class,Insp. Type
3384,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,1.0,Records availability,21 CFR 312.62,data_integrity,1980.0,080295081180,"OAIC,VAI3F","FC,DA"
3385,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,3.0,Inadequate informed consent form,21 CFR 50.25,consent,1980.0,080295081180,"OAIC,VAI3F","FC,DA"
3386,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,1980.0,080295081180,"OAIC,VAI3F","FC,DA"
3387,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,1980.0,080295081180,"OAIC,VAI3F","FC,DA"
3388,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,1980.0,080295081180,"OAIC,VAI3F","FC,DA"
3389,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,14.0,Failure to obtain or document IRB approval,"21 CFR 312.60, 62, 66; 56.103",consent,1980.0,080295081180,"OAIC,VAI3F","FC,DA"


## deficiency percentage per audit

In [114]:
def_perc_of_insp = 1 / finalset.assign(rwn=1).loc[:,['rwn', 'insp_id']].groupby('insp_id').transform('sum')
finalset['def_perc_of_insp'] = def_perc_of_insp

finalset.loc[:,['insp_id', 'def_perc_of_insp']] \
    .sort_values('insp_id') \
    .head(10)
    

,insp_id,def_perc_of_insp
0,000044081800,1.00
1,000044090386,1.00
2,000066011393,1.00
3996,000066051083,1.00
4,000099020397,1.00
3,000099120715,1.00
6,000102120887,0.50
5,000102120887,0.50
7,000113032691,0.25
8,000113032691,0.25


In [118]:
finalset.query('insp_id =="080295081180"') \
    .sort_values('insp_codes')

,site_id,Name,Location,Address,City,State,Cntry,Zip,Insp. Date,Insp. Type,insp_class,insp_codes,deficiency,cfr_reference,mapped_clinical_impact_factor,def_perc_of_insp,year,insp_id
3384,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,1.0,Records availability,21 CFR 312.62,data_integrity,0.083333,1980.0,080295081180
4501,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,1.0,Records availability,21 CFR 312.62,data_integrity,0.083333,1980.0,080295081180
3385,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,3.0,Inadequate informed consent form,21 CFR 50.25,consent,0.083333,1980.0,080295081180
4502,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,3.0,Inadequate informed consent form,21 CFR 50.25,consent,0.083333,1980.0,080295081180
3386,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,0.083333,1980.0,080295081180
4503,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,4.0,Inadequate drug accountability,"21 CFR 312.60, 312.62",primary_endpoints_and_safety,0.083333,1980.0,080295081180
3387,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,0.083333,1980.0,080295081180
4504,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,5.0,Failure to follow investigational plan,21 CFR 312.60,primary_endpoints_and_safety,0.083333,1980.0,080295081180
3388,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,DA,VAI3F,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,0.083333,1980.0,080295081180
4505,80295,"Branscomb, Ben",-,-,Birmingham,AL,USA,00000,08/11/80,FC,OAIC,6.0,Inadequate and inaccurate records,21 CFR 312.62,primary_endpoints,0.083333,1980.0,080295081180


# Save

In [43]:
# Export to csv the final set
finalset.to_csv(file_out)